In [ ]:
## Nos traemos del notebook anterior la clase y las cosas que necesitamos

from importnb import Notebook

# Usamos este contexto para cargar el notebook
with Notebook(include_markdown_docstring=False):
    optimizacion_carteras = __import__('02-optimizacion_carteras_LIMPIO')


# Clases Principales
FinancialAsset = optimizacion_carteras.FinancialAsset
DecisionVariable = optimizacion_carteras.DecisionVariable
PortfolioProblem = optimizacion_carteras.PortfolioProblem
add_method = optimizacion_carteras.add_method

# Funciones de Resolución (Solvers)
solve_qubo_basico_vars = optimizacion_carteras.solve_qubo_basico_vars
solve_qubo_basico_risk = optimizacion_carteras.solve_qubo_basico_risk

# Función para generar problemas aleatorios
create_random_problem = optimizacion_carteras.create_random_problem

# Problemas de ejemplo
problema_random = optimizacion_carteras.problema_random
problema_estatico = optimizacion_carteras.problema_estatico


## Preparación de Datos para Optimización de Cartera QUBO

El objetivo de esta fase es transformar los precios históricos brutos en los tres parámetros clave requeridos por nuestro modelo QUBO (Optimización Cuadrática Binaria Sin Restricciones).

Para ello se utiliza un enfoque de Backtesting, donde la optimización se realiza con datos de un período pasado (In-Sample) para validar su efectividad en un período futuro (Out-of-Sample).

Los tres inputs esenciales que obtendremos de la historia de precios son:



1.   Retorno Esperado ($r_i$): El beneficio esperado de cada activo.
2.   Coste ($c_i$): El precio de compra en el momento de la optimización.
3.   Matriz de Covarianza ($\sigma_{ij}$): El componente de riesgo que mide cómo se mueven los activos entre sí.

El siguiente código Python descarga y calcula estos parámetros utilizando un período de entrenamiento fijo (2023-2025) para simular una decisión de inversión tomada a principios de 2025.

In [ ]:

import yfinance as yf
import numpy as np

# --- 1. Definición de Parámetros de Entrada para Backtesting ---

# Tickers de activos (Sectores y Geografías)
# ^GSPC: S&P 500 (Índice USA) | RY: Royal Bank of Canada (Finanzas Canadá)
# NEE: NextEra Energy (Utilities/Energía USA) | DIS: Walt Disney Co (Entretenimiento)
# BABA: Alibaba (Tech China) | SNEJF: Sony Group Corp (Tech/Electro Japón)

TICKERS = ['^GSPC', 'RY', 'NEE', 'DIS', 'BABA', 'SNEJF']
START_DATE = '2023-01-01'
END_DATE = '2025-01-01'
INTERVALO_ANALISIS = '1d' # Diario


# --- 2. Descarga de Datos Históricos (Datasheet Crudo) ---

datos_crudos = yf.download(TICKERS,
                           start=START_DATE,
                           end=END_DATE,
                           interval=INTERVALO_ANALISIS,
                           auto_adjust=True)

precios = datos_crudos['Close'].dropna()


# --- 3. Derivación de Parámetros Clave para el Modelo (a 2025-01-01) ---

## PARÁMETRO 1: Coste (c_i)

# El coste c_i es el precio más reciente antes de la fecha de inversión (END_DATE).
costes_ci = precios.iloc[-1].rename('Coste ($c_i$)')


## PARÁMETRO 2: Retornos Diarios y Retorno Esperado (r_i)

# Calcular los retornos diarios logarítmicos
retornos = np.log(precios / precios.shift(1)).dropna()

# Multiplicamos por 252 (días de trading en un año) para anualizar.
roi = retornos.mean() * 252
roi.rename('Retorno Esperado ($r_i$ Anualizado)', inplace=True)


## PARÁMETRO 3: Matriz de Covarianza (Sigma_ij)

# La Matriz de Covarianza (Sigma) se calcula a partir de los retornos, anualizada
sigma_matrix = retornos.cov() * 252

print(f"\n- **Fecha de Optimización (Entrada del Modelo):** {END_DATE}")
print(f"\n- **r_i (Retorno Esperado):**\n{roi}")
print(f"\n- **c_i (Coste/Precio):**\n{costes_ci}")
print(f"\n- **Matriz Sigma (Covarianza):**\n{sigma_matrix}")

 ## Evaluación del Rendimiento de la Cartera (Backtesting Post-Inversión)

 Una vez que el modelo de optimización QUBO haya utilizado los datos históricos (hasta 2025-01-01) para seleccionar la cartera óptima de activos ($x_i$), el siguiente paso crucial es el Backtesting.

 Este código tiene como objetivo simular la inversión real midiendo el rendimiento acumulado que esos activos obtuvieron en el mercado en el tiempo posterior a la decisión.

 Al analizar el rendimiento en intervalos de 15 días, 1 mes, 2 meses, etc., podemos confirmar la efectividad de la optimización. Si la cartera seleccionada por el modelo supera consistentemente al mercado o a carteras aleatorias, se valida la calidad de la decisión.

 El código descarga los precios desde el día después de la inversión y calcula la tasa de crecimiento de cada activo en los puntos temporales definidos.

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta

# Fecha de inicio del Backtesting (1  día después de la inversión)
START_BACKTEST = pd.to_datetime(END_DATE) + pd.Timedelta(days=1)

END_BACKTEST_MAX = pd.to_datetime('2025-07-02')

# Definición de los puntos de tiempo para el análisis
TIMEFRAMES = {
    '15 Días': START_BACKTEST + pd.Timedelta(days=15),
    'Mes 1': START_BACKTEST + relativedelta(months=1),
    'Mes 2': START_BACKTEST + relativedelta(months=2),
    'Mes 3': START_BACKTEST + relativedelta(months=3),
    'Mes 4': START_BACKTEST + relativedelta(months=4),
    'Mes 5': START_BACKTEST + relativedelta(months=5),
    'Mes 6': START_BACKTEST + relativedelta(months=6),
}


# 1. Descarga de Datos (Out-of-Sample)

datos_crudos_out = yf.download(
    TICKERS,
    start=START_BACKTEST.strftime('%Y-%m-%d'),
    end=END_BACKTEST_MAX.strftime('%Y-%m-%d'),
    interval=INTERVALO_ANALISIS,
    auto_adjust=True
)

precios_out = datos_crudos_out['Close'].dropna()

# Precio de Cierre el día de la inversión (P_inicial)
P_INICIAL = precios_out.iloc[0]


# 2. Cálculo del Rendimiento por Intervalo

resultados_rendimiento = {}

for label, end_date in TIMEFRAMES.items():
    # Encuentra la fila con la fecha más cercana a end_date
    P_FINAL_SERIE = precios_out.loc[precios_out.index <= end_date].iloc[-1]

    # Rendimiento Acumulado = ((P_final / P_inicial) - 1)
    rendimiento_pct = ((P_FINAL_SERIE / P_INICIAL) - 1)

    resultados_rendimiento[label] = rendimiento_pct.round(2)

# 3. Presentación de Resultados


print("\n Comportamiento del Mercado (Rendimiento Acumulado)")
print(f"Inversión Base (P_inicial): {START_BACKTEST.date()}")
print("-" * 60)


df_resultados = pd.DataFrame(resultados_rendimiento)
df_resultados = df_resultados.transpose()
df_resultados.index.name = 'Intervalo'

print(df_resultados)


In [ ]:
import pandas as pd
import numpy as np
from dimod import ExactSolver
from collections import defaultdict
import math

# -----------------------------------------------------------------------------
# 1. CONFIGURACIÓN: Mapeo de Datos Reales al Modelo
# -----------------------------------------------------------------------------

# Usamos los datos obtenidos de yfinance en las celdas anteriores
assets = TICKERS  # Lista de nombres ['^GSPC', 'RY', ...]

# Convertimos las Series de Pandas a Diccionarios para acceso rápido
prices = costes_ci.to_dict()  # { 'BABA': 83.38, ... }
roi_dict = roi.to_dict()      # { 'BABA': -0.023, ... } (Renombramos para no confundir)
Sigma = sigma_matrix.values   # Convertimos el DataFrame de covarianza a matriz Numpy

# --- Parámetros de Inversión ---
# IMPORTANTE: Definimos un presupuesto C.
# Nota: Como usamos ExactSolver, mantén el presupuesto moderado para no
# generar demasiadas variables binarias (bits), o el solver tardará mucho.
C = 1000.0 

# Hiperparámetros del modelo
alpha = 0.5   # Penalización por desviarse del presupuesto (fuerza a gastar C)
beta  = 1.0   # Aversión al riesgo (peso del término de covarianza)

print(f"Configurando optimización para Presupuesto: ${C}")
print(f"Activos disponibles: {len(assets)}")

# -----------------------------------------------------------------------------
# 2. GENERACIÓN DE VARIABLES BINARIAS (Lotes)
# -----------------------------------------------------------------------------
# Replicamos la lógica de descomposición binaria con los datos reales

lots = []
for asset in assets:
    p = prices[asset]
    
    # Si el precio es mayor que el presupuesto, no podemos comprar ni una unidad
    if p > C:
        continue
        
    qmax = int(C // p)  # Máximo de acciones posibles
    
    if qmax <= 0:
        continue

    # Calcular cuántos bits (potencias de 2) necesitamos
    K = int(math.floor(math.log2(qmax))) 

    for k in range(K + 1):
        units = 2**k
        # Aseguramos no pasarnos del qmax total en la última potencia si fuera necesario, 
        # pero la descomposición estándar 2^k suele ser suficiente para aproximar.
        
        var_name = f"x_{asset}_{k}"
        cost = units * p
        
        # El retorno esperado total de este lote
        # Retorno unitario = ROI anualizado * Precio
        ret_expected = (roi_dict[asset] * p) * units
        
        lots.append({
            "var": var_name,
            "asset": asset,
            "k": k,
            "units": units,
            "price": p,
            "cost": cost,
            "ret": ret_expected
        })

print(f"Total de variables binarias generadas: {len(lots)}")
if len(lots) > 25:
    print("¡ADVERTENCIA! Más de 25 variables puede ser muy lento para ExactSolver.")

# -----------------------------------------------------------------------------
# 3. CONSTRUCCIÓN DE LA MATRIZ QUBO (Q)
# -----------------------------------------------------------------------------
Q = defaultdict(float)
asset_index = {a: i for i, a in enumerate(assets)}

# A) Término de Retorno (Maximizar retorno -> Minimizar negativo)
for row in lots:
    v = row["var"]
    Q[(v, v)] += -row["ret"]

# B) Término de Riesgo (Minimizar Varianza)
for u in lots:
    a_name = u["asset"]
    ia = asset_index[a_name]
    for v in lots:
        b_name = v["asset"]
        ib = asset_index[b_name]
        
        # Covarianza entre activo A y activo B
        cov_val = Sigma[ia, ib]
        
        # Coeficiente: beta * sigma * unidades_u * unidades_v
        coef = beta * cov_val * (u["units"] * v["units"])
        
        uvar, vvar = u["var"], v["var"]
        
        # Sumar al término correspondiente en la matriz triangular superior
        if uvar == vvar:
            Q[(uvar, uvar)] += coef
        elif uvar < vvar: # Solo una vez para pares distintos
            Q[(uvar, vvar)] += 2 * coef # x2 porque la matriz es simétrica
            
# C) Término de Presupuesto (Penalización alpha * (coste_total - C)^2)
# Expansión: alpha * [ sum(c_i^2 x_i) + 2*sum(c_i c_j x_i x_j) - 2*C*sum(c_i x_i) ]
for i, u in enumerate(lots):
    uvar = u["var"]
    cu = u["cost"]
    
    # Diagonal: alpha * cost^2 - 2 * alpha * C * cost
    Q[(uvar, uvar)] += alpha * (cu**2) - 2.0 * alpha * C * cu
    
    # Fuera de diagonal: 2 * alpha * cost_u * cost_v
    for j in range(i + 1, len(lots)):
        v = lots[j]
        vvar = v["var"]
        cv = v["cost"]
        
        # Key ordenada
        key = (uvar, vvar) if uvar <= vvar else (vvar, uvar)
        Q[key] += 2.0 * alpha * cu * cv

# -----------------------------------------------------------------------------
# 4. RESOLUCIÓN Y VISUALIZACIÓN DE RESULTADOS
# -----------------------------------------------------------------------------

# Resolver QUBO
print("Resolviendo QUBO (esto puede tardar unos segundos)...")
solver = ExactSolver()
sampleset = solver.sample_qubo(Q)
best = sampleset.first
solution = best.sample
energy = best.energy

print("Mejor energía encontrada:", energy)

# Filtrar variables activas
active_vars = [v for v, val in solution.items() if val == 1]

# Reconstruir cartera
units_by_asset = {a: 0 for a in assets}
cost_by_asset  = {a: 0.0 for a in assets}
ret_by_asset   = {a: 0.0 for a in assets}
lot_by_var = {row["var"]: row for row in lots}

for v in active_vars:
    info = lot_by_var[v]
    a = info["asset"]
    units_by_asset[a] += info["units"]
    cost_by_asset[a]  += info["cost"]
    ret_by_asset[a]   += info["ret"]

total_cost = sum(cost_by_asset.values())
total_ret  = sum(ret_by_asset.values())

# Calcular riesgo resultante
q_vec = np.array([units_by_asset[a] for a in assets], dtype=float)
final_risk = float(q_vec.T @ Sigma @ q_vec)

# Crear DataFrame de resumen
summary_df = pd.DataFrame({
    "Ticker": assets,
    "Unidades": [units_by_asset[a] for a in assets],
    "Precio Unit.": [prices[a] for a in assets],
    "Inversión ($)": [cost_by_asset[a] for a in assets],
    "Retorno Esp. ($)": [ret_by_asset[a] for a in assets],
})

# Formateo visual
print("\n=== CARTERA ÓPTIMA (QUBO) BASADA EN DATOS REALES ===")
display(summary_df[summary_df['Unidades'] > 0]) # Solo mostramos lo que compramos

print("-" * 40)
print(f"Presupuesto Objetivo: ${C}")
print(f"Inversión Total:      ${total_cost:.2f}")
print(f"Retorno Esperado:     ${total_ret:.2f}")
print(f"Riesgo (Varianza):    {final_risk:.6f}")

if abs(total_cost - C) < (C * 0.05):
    print("Estado: Presupuesto utilizado correctamente.")
else:
    print("Estado: Inversión baja. Intenta aumentar 'alpha' o revisar precios vs presupuesto.")

# Benchmark para código QUBO sin variables de holgura (con igualdad en la restricción del presupuesto).

In [ ]:
import math
import numpy as np
from dimod import ExactSolver
from tabulate import tabulate

# ---------------------------------------------------------
# 1. Definición de Clases (Infraestructura del Modelo)
# ---------------------------------------------------------
class FinancialAsset:
    def __init__(self, name, price, roi, variance=0):
        self.name = name
        self.price = price
        self.roi = roi
        self.variance = variance

class DecisionVariable:
    def __init__(self, parent_name, quantity, cost, value):
        self.parent_name = parent_name
        self.quantity = quantity
        self.cost = cost
        self.value = value

class PortfolioProblem:
    def __init__(self, max_budget):
        self.max_budget = max_budget
        self.assets = {}
        self.covariances = {}
        self.variables = []

    def add_asset(self, name, price, roi, variance=0):
        self.assets[name] = FinancialAsset(name, price, roi, variance)

    def add_covariance(self, asset1_name, asset2_name, value):
        key = tuple(sorted((asset1_name, asset2_name)))
        self.covariances[key] = value

    def get_asset_risk(self, name1, name2):
        if name1 == name2:
            return self.assets[name1].variance
        key = tuple(sorted((name1, name2)))
        return self.covariances.get(key, 0.0)

    def create_variables_binary_simple(self):
        """Crea variables usando descomposición binaria (x_i,k)."""
        for asset in self.assets.values():
            if asset.price > self.max_budget:
                continue
            max_units = math.floor(self.max_budget / asset.price)
            if max_units < 1: continue
            
            k = math.floor(math.log2(max_units))
            for i in range(k + 1):
                count = 2**i
                cost = count * asset.price
                val = cost * asset.roi # Retorno esperado del paquete
                self.variables.append(DecisionVariable(asset.name, count, cost, val))

    def get_interaction_risk(self, var_i, var_j):
        """Calcula el riesgo ponderado por las cantidades de los paquetes."""
        cov = self.get_asset_risk(var_i.parent_name, var_j.parent_name)
        return var_i.quantity * var_j.quantity * cov

    def create_qubo_equality_constraint(self, alpha, beta):
        """Genera matriz QUBO con restricción de IGUALDAD (sin variables slack)."""
        Q = {}
        C = self.max_budget
        n_vars = len(self.variables)

        # Inicializar Q
        for i in range(n_vars):
            for j in range(i, n_vars):
                Q[(i, j)] = 0.0

        for i in range(n_vars):
            var_i = self.variables[i]
            
            # 1. Retorno (Lineal -> Diagonal)
            # Maximizar retorno => Minimizar -Retorno
            Q[(i, i)] += -var_i.value

            for j in range(i, n_vars):
                var_j = self.variables[j]
                
                # 2. Presupuesto (Igualdad estricta)
                # alpha * (Sum cost - C)^2
                ci = var_i.cost
                cj = var_j.cost
                
                if i == j:
                    Q[(i, i)] += alpha * (ci**2 - 2 * C * ci)
                else:
                    Q[(i, j)] += 2 * alpha * ci * cj

                # 3. Riesgo
                # beta * Sum sigma_ij * q_i * q_j
                risk_term = beta * self.get_interaction_risk(var_i, var_j)
                
                if i != j:
                    risk_term *= 2.0 # Matriz simétrica
                
                Q[(i, j)] += risk_term
        return Q

    def print_solution(self, solution_vector):
        print("\n=== Solución Óptima ===")
        summary = {}
        total_inv = 0
        total_ret = 0

        for i, selected in enumerate(solution_vector):
            if selected > 0:
                var = self.variables[i]
                if var.parent_name not in summary:
                    summary[var.parent_name] = {'u': 0, 'c': 0.0, 'v': 0.0}
                
                summary[var.parent_name]['u'] += var.quantity
                summary[var.parent_name]['c'] += var.cost
                summary[var.parent_name]['v'] += var.value
                total_inv += var.cost
                total_ret += var.value

        data = []
        for name, info in summary.items():
            pct = (info['c'] / total_inv) * 100 if total_inv > 0 else 0
            data.append([name, info['u'], f"${info['c']:.2f}", f"{pct:.1f}%", f"${info['v']:.2f}"])

        print(tabulate(data, headers=["Activo", "Unidades", "Inversión", "% Cartera", "Retorno Esp."]))
        print(f"\nTotal Invertido: ${total_inv:.2f} (Presupuesto: ${self.max_budget})")
        print(f"Retorno Esperado Total: ${total_ret:.2f}")
# ---------------------------------------------------------
# 2. Ingesta de Datos (DINÁMICA - Conectada a Pandas)
# ---------------------------------------------------------

# Configuración de Hiperparámetros
PRESUPUESTO = 1000.0
ALPHA = 0.5  # Penalización presupuesto
BETA = 1.0   # Aversión al riesgo

# Instanciamos el problema
portfolio = PortfolioProblem(max_budget=PRESUPUESTO)

# --- CAPTURA DE DATOS VIVOS (Desde celdas anteriores) ---
# Se asume que existen las variables: TICKERS, costes_ci, roi, sigma_matrix

# Lista de activos
assets_list = TICKERS 

# Convertimos a diccionarios/numpy para iterar fácilmente
prices_dict = costes_ci.to_dict()
roi_dict = roi.to_dict()
sigma_vals = sigma_matrix.values # Matriz numpy pura

print(f"Cargando datos para {len(assets_list)} activos...")

# a) Cargar Activos (Coste y ROI)
for asset in assets_list:
    p = prices_dict[asset]
    r = roi_dict[asset]
    # Agregamos al portfolio
    portfolio.add_asset(asset, price=p, roi=r)

# b) Cargar Covarianzas (Sigma)
# Iteramos sobre la matriz de covarianza y llenamos el modelo
for i, asset_a in enumerate(assets_list):
    for j, asset_b in enumerate(assets_list):
        val = sigma_vals[i, j]
        
        # Agregamos la covarianza entre A y B
        portfolio.add_covariance(asset_a, asset_b, val)
        
        # Si es el mismo activo (diagonal), actualizamos su varianza interna
        if i == j:
             portfolio.assets[asset_a].variance = val

print("Datos cargados correctamente.")

# ---------------------------------------------------------
# 3. Ejecución del Flujo
# ---------------------------------------------------------

print("Generando variables binarias...")
portfolio.create_variables_binary_simple()
print(f"Variables generadas: {len(portfolio.variables)}")

print(f"Creando QUBO (Alpha={ALPHA}, Beta={BETA})...")
Q = portfolio.create_qubo_equality_constraint(alpha=ALPHA, beta=BETA)

print("Resolviendo con ExactSolver...")
solver = ExactSolver()
response = solver.sample_qubo(Q)
best_sample = response.first.sample
solution_vector = [best_sample[i] for i in range(len(portfolio.variables))]

portfolio.print_solution(solution_vector)

# Benchmark para crear paquetes de acciones para un activo financiero como un múltiplo de las acciones individuales (multiplo a definir)

In [ ]:
import math
from dimod import ExactSolver
from tabulate import tabulate

# ---------------------------------------------------------
# 1. CLASES DEL MODELO QUBO
# ---------------------------------------------------------

class FinancialAsset:
    def __init__(self, name, price, roi, variance=0):
        self.name = name
        self.price = price
        self.roi = roi
        self.variance = variance

class DecisionVariable:
    def __init__(self, parent_name, quantity, cost, value):
        self.parent_name = parent_name
        self.quantity = quantity
        self.cost = cost
        self.value = value

class PortfolioProblem:
    def __init__(self, max_budget):
        self.max_budget = max_budget
        self.assets = {}
        self.covariances = {}
        self.variables = []

    def add_asset(self, name, price, roi, variance=0):
        self.assets[name] = FinancialAsset(name, price, roi, variance)

    def add_covariance(self, asset1_name, asset2_name, value):
        key = tuple(sorted((asset1_name, asset2_name)))
        self.covariances[key] = value

    def get_asset_risk(self, name1, name2):
        if name1 == name2:
            return self.assets[name1].variance
        key = tuple(sorted((name1, name2)))
        return self.covariances.get(key, 0.0)

    # --- MÉTODO CLAVE: Generación de Variables por Lotes Fijos ---
    def create_variables_fixed_batch(self, batch_size=5):
        """
        Crea variables para comprar acciones en múltiplos exactos de batch_size.
        Ej: batch_size=5 -> Variables para 5, 10, 20... acciones.
        """
        print(f"--- Configuración: Lotes de {batch_size} acciones ---")
        
        for asset in self.assets.values():
            if asset.price > self.max_budget:
                continue
            
            # Coste del lote mínimo
            batch_cost = batch_size * asset.price
            
            # Si el lote mínimo excede el presupuesto, se descarta este activo
            if batch_cost > self.max_budget:
                continue

            # Cuántos lotes caben en el presupuesto?
            max_batches = math.floor(self.max_budget / batch_cost)
            
            # Descomposición binaria del número de lotes
            k = math.floor(math.log2(max_batches))

            for i in range(k + 1):
                num_batches = 2**i 
                real_units = num_batches * batch_size
                cost = real_units * asset.price
                val = cost * asset.roi
                
                self.variables.append(DecisionVariable(asset.name, real_units, cost, val))

    def get_interaction_risk(self, var_i, var_j):
        cov = self.get_asset_risk(var_i.parent_name, var_j.parent_name)
        return var_i.quantity * var_j.quantity * cov

    def create_qubo_equality_constraint(self, alpha, beta):
        Q = {}
        C = self.max_budget
        n_vars = len(self.variables)
        
        # Inicializar Q
        for i in range(n_vars):
            for j in range(i, n_vars): Q[(i, j)] = 0.0

        for i in range(n_vars):
            var_i = self.variables[i]
            
            # 1. Retorno (Minimizar negativo para maximizar ganancia)
            Q[(i, i)] += -var_i.value
            
            # 2. Presupuesto (Diagonal): alpha * (cost^2 - 2*C*cost)
            Q[(i, i)] += alpha * (var_i.cost**2 - 2 * C * var_i.cost)

            for j in range(i, n_vars):
                var_j = self.variables[j]
                
                # 2. Presupuesto (Fuera diagonal): 2 * alpha * cost_i * cost_j
                if i != j:
                    Q[(i, j)] += 2 * alpha * var_i.cost * var_j.cost
                
                # 3. Riesgo: beta * sigma * q_i * q_j
                risk_term = beta * self.get_interaction_risk(var_i, var_j)
                if i != j: risk_term *= 2.0 # Matriz simétrica
                Q[(i, j)] += risk_term
        return Q

    # --- Visualización Agrupada ---
    def print_solution(self, solution_vector):
        print("\n=== Solución Óptima ===")
        summary = {}
        total_inv = 0.0
        total_ret = 0.0

        for i, selected in enumerate(solution_vector):
            if selected > 0:
                var = self.variables[i]
                name = var.parent_name
                
                if name not in summary: 
                    summary[name] = {'u': 0, 'c': 0.0, 'v': 0.0}
                
                summary[name]['u'] += var.quantity
                summary[name]['c'] += var.cost
                summary[name]['v'] += var.value
                
                total_inv += var.cost
                total_ret += var.value

        table_data = []
        for name, info in summary.items():
            pct = (info['c'] / total_inv * 100) if total_inv > 0 else 0
            row = [
                name, 
                info['u'], 
                f"${info['c']:.2f}", 
                f"{pct:.1f}%", 
                f"${info['v']:.2f}"
            ]
            table_data.append(row)

        print(tabulate(table_data, headers=["Activo", "Unidades", "Inversión", "% Cartera", "Retorno Esp."]))
        print(f"\nTotal Invertido: ${total_inv:.2f} (Presupuesto: ${self.max_budget})")
        print(f"Retorno Esperado Total: ${total_ret:.2f}")


# ---------------------------------------------------------
# 2. EJECUCIÓN DEL MODELO (Usando variables existentes)
# ---------------------------------------------------------

# Configuración
PRESUPUESTO = 1000.0
BATCH_SIZE = 5     # Tamaño del lote (multiplos de 5)
ALPHA = 1.0        # Importancia de gastar todo el presupuesto
BETA = 1.0         # Importancia de minimizar riesgo

# Instanciamos el problema
portfolio = PortfolioProblem(max_budget=PRESUPUESTO)

# Usamos TICKERS (definido en tu bloque anterior)
tickers_existentes = TICKERS 

print(f"--- Alimentando modelo con datos en memoria ({len(tickers_existentes)} activos) ---")

# 1. Cargar Activos (Precio y ROI desde Pandas Series)
for t in tickers_existentes:
    # costes_ci y roi son Pandas Series generadas en tu bloque anterior
    try:
        p = float(costes_ci[t])
        r = float(roi[t])
        portfolio.add_asset(t, price=p, roi=r)
    except KeyError:
        print(f"Advertencia: Datos no encontrados para {t}")

# 2. Cargar Covarianzas (desde Pandas DataFrame sigma_matrix)
for t1 in tickers_existentes:
    for t2 in tickers_existentes:
        try:
            val = float(sigma_matrix.loc[t1, t2])
            portfolio.add_covariance(t1, t2, val)
            
            # Si es diagonal, guardamos varianza interna
            if t1 == t2 and t1 in portfolio.assets:
                portfolio.assets[t1].variance = val
        except KeyError:
            continue

# 3. Crear Variables (Lotes de 5)
portfolio.create_variables_fixed_batch(batch_size=BATCH_SIZE)
print(f"Variables binarias generadas: {len(portfolio.variables)}")

# 4. Resolver QUBO
print("Resolviendo...")
Q = portfolio.create_qubo_equality_constraint(alpha=ALPHA, beta=BETA)
solver = ExactSolver()
response = solver.sample_qubo(Q)

# 5. Mostrar Resultados
best_sample = response.first.sample
solution_vector = [best_sample.get(i, 0) for i in range(len(portfolio.variables))]
portfolio.print_solution(solution_vector)

# Benchmark para limitar en % la inversión en un solo activo financiero dentro del modelo QUBO.

In [ ]:
import math
from dimod import ExactSolver
from tabulate import tabulate

# ---------------------------------------------------------
# 1. CLASES DEL MODELO
# ---------------------------------------------------------

class FinancialAsset:
    def __init__(self, name, price, roi, variance=0):
        self.name = name
        self.price = price
        self.roi = roi
        self.variance = variance

class DecisionVariable:
    def __init__(self, parent_name, quantity, cost, value):
        self.parent_name = parent_name
        self.quantity = quantity
        self.cost = cost
        self.value = value

class PortfolioProblem:
    def __init__(self, max_budget):
        self.max_budget = max_budget
        self.assets = {}
        self.covariances = {}
        self.variables = []

    def add_asset(self, name, price, roi, variance=0):
        self.assets[name] = FinancialAsset(name, price, roi, variance)

    def add_covariance(self, asset1_name, asset2_name, value):
        key = tuple(sorted((asset1_name, asset2_name)))
        self.covariances[key] = value

    def get_asset_risk(self, name1, name2):
        if name1 == name2:
            return self.assets[name1].variance
        key = tuple(sorted((name1, name2)))
        return self.covariances.get(key, 0.0)

    
   # --- MÉTODO CORREGIDO: Límite Estricto (Sin pasarse) ---
    def create_variables_limited(self, max_weight_pct=1.0):
        """
        Crea variables binarias asegurando que LA SUMA TOTAL de variables
        para un activo NUNCA supere el límite de peso establecido.
        """
        limit_money = self.max_budget * max_weight_pct
        print(f"--- Configuración: Compra Libre con Límite Estricto ---")
        print(f"--- Restricción: Máximo {int(max_weight_pct*100)}% (${limit_money:.2f}) ---")
        
        for asset in self.assets.values():
            if asset.price > self.max_budget:
                continue

            # 1. Calcular el techo de acciones (Límite estricto)
            current_limit_money = min(self.max_budget, limit_money)
            max_units = math.floor(current_limit_money / asset.price)
            
            if max_units < 1:
                continue

            # 2. Generación Inteligente de Variables (Truncada)
            # Ejemplo: Si max_units es 2.
            # Intento 1 (2^0): Añado var de 1. (Suma actual: 1). Queda espacio para 1.
            # Intento 2 (2^1): 2. ¿1+2 > 2? Sí. 
            # Corrección: En vez de añadir 2, añado el resto (2-1=1).
            # Resultado: Variables [1, 1]. Suma máxima posible = 2.
            
            current_count = 0
            k = math.floor(math.log2(max_units))

            for i in range(k + 1):
                units = 2**i
                
                # Si añadir esta potencia hace que nos pasemos del tope...
                if current_count + units > max_units:
                    # ...añadimos solo lo que falta para llegar al tope exacto
                    remainder = max_units - current_count
                    if remainder > 0:
                        cost = remainder * asset.price
                        val = cost * asset.roi
                        self.variables.append(DecisionVariable(asset.name, remainder, cost, val))
                        current_count += remainder
                    break # Terminamos con este activo
                
                # Si cabe entera, la añadimos normal
                cost = units * asset.price
                val = cost * asset.roi
                self.variables.append(DecisionVariable(asset.name, units, cost, val))
                current_count += units

    def get_interaction_risk(self, var_i, var_j):
        cov = self.get_asset_risk(var_i.parent_name, var_j.parent_name)
        return var_i.quantity * var_j.quantity * cov

    def create_qubo_equality_constraint(self, alpha, beta):
        Q = {}
        C = self.max_budget
        n_vars = len(self.variables)
        
        for i in range(n_vars):
            for j in range(i, n_vars): Q[(i, j)] = 0.0

        for i in range(n_vars):
            var_i = self.variables[i]
            Q[(i, i)] += -var_i.value # Retorno
            Q[(i, i)] += alpha * (var_i.cost**2 - 2 * C * var_i.cost) # Presupuesto

            for j in range(i, n_vars):
                var_j = self.variables[j]
                if i != j:
                    Q[(i, j)] += 2 * alpha * var_i.cost * var_j.cost
                
                risk_term = beta * self.get_interaction_risk(var_i, var_j)
                if i != j: risk_term *= 2.0
                Q[(i, j)] += risk_term
        return Q

    def print_solution(self, solution_vector):
        print("\n=== Solución Óptima (Límite 30%) ===")
        summary = {}
        total_inv = 0.0
        total_ret = 0.0

        for i, selected in enumerate(solution_vector):
            if selected > 0:
                var = self.variables[i]
                name = var.parent_name
                if name not in summary: summary[name] = {'u': 0, 'c': 0.0, 'v': 0.0}
                
                summary[name]['u'] += var.quantity
                summary[name]['c'] += var.cost
                summary[name]['v'] += var.value
                total_inv += var.cost
                total_ret += var.value

        table_data = []
        for name, info in summary.items():
            pct = (info['c'] / total_inv * 100) if total_inv > 0 else 0
            row = [name, info['u'], f"${info['c']:.2f}", f"{pct:.1f}%", f"${info['v']:.2f}"]
            table_data.append(row)

        print(tabulate(table_data, headers=["Activo", "Unidades", "Inversión", "% Cartera", "Retorno Esp."]))
        print(f"\nTotal Invertido: ${total_inv:.2f} (Presupuesto: ${self.max_budget})")
        print(f"Retorno Esperado Total: ${total_ret:.2f}")


# ---------------------------------------------------------
# 2. EJECUCIÓN (Usando los datos en memoria)
# ---------------------------------------------------------

# Configuración
PRESUPUESTO = 1000.0
MAX_WEIGHT = 0.30   # Máximo 30% del presupuesto por empresa
ALPHA = 1.0
BETA = 1.0

portfolio = PortfolioProblem(max_budget=PRESUPUESTO)

# Usamos los datos (TICKERS, costes_ci, roi, sigma_matrix) del bloque anterior
tickers_existentes = TICKERS 

print(f"--- Alimentando modelo con {len(tickers_existentes)} activos ---")

# Cargar Datos
for t in tickers_existentes:
    try:
        portfolio.add_asset(t, price=float(costes_ci[t]), roi=float(roi[t]))
    except KeyError: pass

for t1 in tickers_existentes:
    for t2 in tickers_existentes:
        try:
            val = float(sigma_matrix.loc[t1, t2])
            portfolio.add_covariance(t1, t2, val)
            if t1 == t2 and t1 in portfolio.assets:
                portfolio.assets[t1].variance = val
        except KeyError: continue

# 1. Crear Variables (Límite 30%, sin lotes forzados)
portfolio.create_variables_limited(max_weight_pct=MAX_WEIGHT)
print(f"Variables binarias generadas: {len(portfolio.variables)}")

# 2. Resolver
print("Resolviendo QUBO...")
Q = portfolio.create_qubo_equality_constraint(alpha=ALPHA, beta=BETA)
solver = ExactSolver()
response = solver.sample_qubo(Q)

# 3. Resultados
best_sample = response.first.sample
solution_vector = [best_sample.get(i, 0) for i in range(len(portfolio.variables))]
portfolio.print_solution(solution_vector)